# Trend Lines and Channels Detection

In [13]:
import ccxt
import pandas as pd

exchange = ccxt.binance()
exchange.load_markets()

'''symbols variabl contains all the symbols of the exchange '''
symbols = exchange.symbols

# ticker = 'BTC-USDT'
# timeframe ='4h'
def ccxt_data(symbol='HNT/USDT', timeframe ='4h', limit=9000):

    """ use global to make a variable accesible by oher functions
    and always declare it before using it  """
    global df

    '''if you dont do timeframe=timeframe it will not work 
    but in symbol it works !!'''
    # fetch returns a list
    bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

    df = pd.DataFrame(bars[:], columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    # Time is in unix format
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df.set_index('time', inplace=True)
    
    return df


In [14]:
import pandas as pd
ccxt_data()
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,open,high,low,close,volume
0,13.052,13.169,12.900,12.994,46188.359
1,12.994,13.198,12.851,13.180,59255.691
2,13.180,13.587,13.175,13.469,75437.475
3,13.468,13.572,13.297,13.336,65350.609
4,13.366,13.443,13.153,13.258,51631.748
5,13.262,13.426,12.971,13.373,50667.366
6,13.363,13.750,13.183,13.604,114208.000
7,13.586,13.699,12.800,12.972,233048.696
8,12.950,13.250,12.694,13.116,114542.215
9,13.109,13.394,13.048,13.225,83792.579


In [15]:
dfpl = df[:]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.show()

In [16]:
import numpy as np
from matplotlib import pyplot
backcandles= 50
wind = 5

candleid = 880

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])
for i in range(candleid-backcandles, candleid+1, wind):
    minim = np.append(minim, df.low.iloc[i:i+wind].min())
    xxmin = np.append(xxmin, df.low.iloc[i:i+wind].idxmin())
for i in range(candleid-backcandles, candleid+1, wind):
    maxim = np.append(maxim, df.high.loc[i:i+wind].max())
    xxmax = np.append(xxmax, df.high.iloc[i:i+wind].idxmax())
slmin, intercmin = np.polyfit(xxmin, minim,1)
slmax, intercmax = np.polyfit(xxmax, maxim,1)

dfpl = df[candleid-backcandles:candleid+backcandles]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))

# Fitting intercepts to meet highest or lowest candle point in time slice

In [17]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to meet highest or lowest candle point in time slice
adjintercmin = df.low.loc[candleid-backcandles:candleid].min() - slmin*df.low.iloc[candleid-backcandles:candleid].idxmin()
adjintercmax = df.high.loc[candleid-backcandles:candleid].max() - slmax*df.high.iloc[candleid-backcandles:candleid].idxmax()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.show()

# Fitting intercepts to wrap price candles

In [18]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to wrap price candles
adjintercmax = (df.high.iloc[xxmax] - slmax*xxmax).max()
adjintercmin = (df.low.iloc[xxmin] - slmin*xxmin).min()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.show()

# backcandles time window more dynamic

In [19]:
import numpy as np
from matplotlib import pyplot
backcandles= 40 # 6*8
brange = 10 # backcandles//4 #should be less than backcandles
wind = 6

candleid = 500

optbackcandles= backcandles
sldiff = 1000

for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.high.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.high.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    if(abs(slmin-slmax)<sldiff):
        sldiff = abs(slmin-slmax)
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

adjintercmax = (df.high.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))
fig.show()

30
31
41
43
49


In [20]:
import numpy as np
from matplotlib import pyplot
backcandles= 100
brange = 50 #should be less than backcandles
wind = 5

candleid = 999

optbackcandles= backcandles
sldiff = 100
sldist = 999
for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.high.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.high.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    dist = (slmax*candleid + intercmax)-(slmin*candleid + intercmin)
    if(dist<sldist): #abs(slmin-slmax)<sldiff and
        #sldiff = abs(slmin-slmax)
        sldist = dist
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        
print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

adjintercmax = (df.high.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))
fig.show()

72
